# テストの実行

In [1]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [2]:
TESTSET_DIR = os.path.join('/home', 'jupyter', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jupyter', 'work')

## マスタの読み込み

In [3]:
pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_test_pre.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '02_test_sys.csv'), usecols=['seed'])
stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_test_stress.csv'), usecols=['seed'])

## 実行するロジックの指定

In [4]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'min_bypass'

In [5]:
def solve(seed, N, M, D, K, problem, prog_path):
    start_time = time.perf_counter()
    
    command_str = 'echo {} | {}'.format(problem, prog_path)
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    #print('{}'.format(prob_id))    
    return (seed, N, M, D, K, e_time, res)

In [6]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name+'.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=9) as executor:
        for _, row in testset_df.iterrows():
            seed, N, M, D, K, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  N=N, M=M, D=D, K=K, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, N, M, D, K, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(N)
        solve_result.append(M)
        solve_result.append(D)
        solve_result.append(K)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # コスト
            cost = int(res.stderr.decode('utf-8').split()[0].replace('Cost=', ''))

            # 非連結なノードペア数
            discon_cnt = int(res.stderr.decode('utf-8').split()[1].replace('DisconCnt=', ''))

            # 工事辺数のオーバー
            over_k_cnt = int(res.stderr.decode('utf-8').split()[2].replace('OverK=', ''))
            
            # 迂回路中の辺の数
            in_bypass_cnt = int(res.stderr.decode('utf-8').split()[3].replace('InBypass=', ''))
                       
        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(cost)
        solve_result.append(discon_cnt)
        solve_result.append(over_k_cnt)
        solve_result.append(in_bypass_cnt)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'N', 'M', 'D', 'K', 'time', 'cost', 'discon_cnt', 'over_k_cnt', 'in_bypass_cnt']

    return result_df

In [7]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [8]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'cost_mean': [int(np.mean(test_result_df['cost']))],
            'cost_min': [min(test_result_df['cost'])],
            'cost_max': [max(test_result_df['cost'])],

            'discon_cnt_mean': [np.mean(test_result_df['discon_cnt'])],
            'discon_cnt_max': [max(test_result_df['discon_cnt'])],

            'over_k_cnt_max': [max(test_result_df['over_k_cnt'])],

            'in_bypass_cnt_mean': [np.mean(test_result_df['in_bypass_cnt'])],
            'in_bypass_cnt_max': [max(test_result_df['in_bypass_cnt'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    summary_all_df = summary_all_df[['tag', 'testset', 'time_mean', 
                                     'cost_mean', 'cost_min', 'cost_max',
                                     'discon_cnt_mean', 'discon_cnt_max',
                                     'over_k_cnt_max',
                                     'in_bypass_cnt_mean', 'in_bypass_cnt_max',
                                     'time_max']]
    
    return summary_all_df

In [15]:
PROG_NAME_LIST = ['main']
#PROG_NAME_LIST = ['main', 'main_off']

#TEST_LIST = ['00_test_mini']
#testset_name = '00_test_mini'
#testset_name = '01_test_pre'
#testset_name = '02_test_sys'
testset_name = '03_test_stress'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-01-31 13:32:49,945: Start
2023-01-31 14:03:00,907: finish!


In [16]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,time_max
0,min_bypass,01_test_pre,1608,15282266,3376085,46639481,0.0,0,0,1.66,34,4114


In [17]:
# sys test
summary_df = pd.DataFrame()

if testset_name == '02_test_sys' or testset_name == '03_test_stress':
    result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,time_max
0,min_bypass,02_test_sys,1610,19597465,3039426,123799863,0.0,0,0,6.524,222,6643


In [18]:
# stress test
summary_df = pd.DataFrame()

if testset_name == '03_test_stress':
    result_sub_df = pd.merge(stress_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '03_test_stress'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,time_max
0,min_bypass,03_test_stress,1621,19497421,2968597,132673042,0.0,0,0,6.1903,229,7048


In [19]:
result_df.sort_values('cost')

,testset,seed,N,M,D,K,time,cost,discon_cnt,over_k_cnt,in_bypass_cnt
7917,03_test_stress,7917,890,2619,30,93,3043,2968597,0,0,0
7547,03_test_stress,7547,620,1825,30,63,1418,3027730,0,0,0
9402,03_test_stress,9402,973,2860,30,105,4537,3039426,0,0,0
2571,03_test_stress,2571,963,2830,30,119,3578,3064074,0,0,0
9511,03_test_stress,9511,936,2760,30,144,3814,3076218,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7125,03_test_stress,7125,663,1129,5,227,2156,123799863,0,0,166
5553,03_test_stress,5553,539,917,5,264,1999,127107969,0,0,112
267,03_test_stress,267,670,1120,5,247,3005,128034530,0,0,172
32,03_test_stress,32,517,866,5,310,2429,130428759,0,0,115


# 結果ログの保存

In [20]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)